In [1]:
# Básicas:
from time import sleep
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import os
from dotenv import load_dotenv
import json
import re
from dateutil.relativedelta import relativedelta

# Web Scraping:
import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException

# Captcha sol:
from openai import OpenAI
import sounddevice as sd
from scipy.io.wavfile import write
from dotenv import load_dotenv

# 0. Funciones Especiales

## Captcha

In [2]:
def record_audio(duration=15, sample_rate=44100):
    print("Recording...")

    # Record audio
    audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=2, dtype=np.int16)
    sd.wait()

    print("Recording finished.")

    return audio_data

In [3]:
def save_audio(audio_data, file_path="recorded_audio.mp3", sample_rate=44100):
    print("Saving audio...")

    # Save audio to file using scipy.io.wavfile.write
    write(file_path, sample_rate, audio_data)

    print(f"Audio saved to {file_path}")

## Extracción de datos de tipo texto con try except

In [4]:
def extraccion_datos_try_except_compleja(soup, tags_attrs, elementos_a_encontrar = 'lista_completa'): # Si pasamos el parametro lista completa cogeremos todos los elementos 
                                                                                    #que encuetre con find_all
    
    #Hacemos todas las busquedas simples necesrias con el metodo .find y si falla devolvemos tantos nans como elementos buscabamos
    for tag, attrs in tags_attrs[:-1]:
        try:
            soup = soup.find(tag, attrs)
        except:
            datos = np.nan if elementos_a_encontrar == 'lista_completa' else [np.nan] * len(elementos_a_encontrar)

            return datos

    #Realizamos la busqueda con el find_all
    datos = []
    tag, attrs = tags_attrs[-1]
    
    if elementos_a_encontrar == 'lista_completa':
        try:
            datos = soup.find_all(tag, attrs)
            datos = [dato.get_text(strip = True) for dato in datos]
        except:
            datos = np.nan
        
        return datos
        
    for elemento in elementos_a_encontrar:
        try:
            texto = soup.find_all(tag, attrs)[elemento].get_text(strip = True)
        except:
            texto = np.nan
        
        datos.append(texto)

    return datos

In [5]:
def extraccion_datos_try_except_simple(soup, tags_attrs):
    try:
        for idx, (tag, attrs) in enumerate(tags_attrs):
            dato = soup.find(tag, attrs) if idx == 0 else dato.find(tag, attrs)
        return dato.get_text(strip = True)
    
    except:
        return np.nan

In [6]:
def extraccion_datos_try_except(soup, tag, attrs):
    try:
        dato = soup.find(tag, attrs).get_text(strip = True)
        return dato
    except:
        return np.nan

## Guardado de datos

In [7]:
def guardar_datos(portal, data, nombres_columnas, ruta_datos):
    try:
        df = pd.read_csv(ruta_datos + f'datos_{portal}.csv')

    except FileNotFoundError:
        df = pd.DataFrame(columns=nombres_columnas)
    
    nuevas_filas = pd.DataFrame(data, columns=nombres_columnas)
    
    df = pd.concat([df, nuevas_filas], ignore_index=True) 
    
    df.to_csv(ruta_datos + f'datos_{portal}.csv', index=False, errors='ignore')

In [8]:
def date_matching(date):
    match = re.search(r'(\d{1,2})/(\d{1,2})/(\d{4})|(\d+)\s*(minutos|hora|dia|semana|mes)', date)
    
    if match:
        if match.group(1):
            day = int(match.group(1))
            month = int(match.group(2))
            year = int(match.group(3))
            return relativedelta(day=day, month=month, year=year)
        else:
            cantidad = int(match.group(4))
            unidad = match.group(5)
        
            if unidad == 'minutos':
                return relativedelta(minutes=cantidad)
            elif unidad == 'hora':
                return relativedelta(hours=cantidad)
            elif unidad == 'dia':
                return relativedelta(days=cantidad)
            elif unidad == 'semana':
                return relativedelta(weeks=cantidad)
            elif unidad == 'mes':
                return relativedelta(months=cantidad)
    
    return relativedelta()

In [9]:
def procesar_datos_nuevos(df, portal, ruta_datos):
    #Pasamos todos los datos de fechas a cadena de texto para poder procesarlos
    df['fecha_scrapeo'] = df['fecha_scrapeo'].apply(lambda x : str(x))

    #Datetime
    df['fecha_scrapeo'] = pd.to_datetime(df['fecha_scrapeo'], format='%Y-%m-%d', errors='coerce').dt.date

    #Separamos las ofertas por las scrapeadas recientemente y las antiguas
    ultima_fecha = df['fecha_scrapeo'].unique().max()
    datos_nuevos = df[df['fecha_scrapeo'] == ultima_fecha]
    datos_antiguos = df.drop(datos_nuevos.index)

    #Si existe alguna oferta repetida, eliminamos el registro de la mas antigua
    try:
        merged_df = pd.merge(datos_nuevos, datos_antiguos, how = 'outer', on = ['titulo', 'empresa', 'descripcion'], indicator = True)
    
    except:
        merged_df = pd.merge(datos_nuevos, datos_antiguos, how = 'outer', on = ['titulo', 'descripcion'], indicator = True)
        
    datos_nuevos = merged_df[merged_df['_merge'] == 'left_only'].drop('_merge', axis = 1)

    #Eliminamos las columnas mergeadas del dataset antiguo y renombramos las de los datos nuevos
    columnas_eliminar = [columna for columna in datos_nuevos.columns if columna.endswith('_y')]
    datos_nuevos = datos_nuevos.drop(columnas_eliminar, axis = 1)

    datos_nuevos.columns = [columna[:-2] if columna.endswith('_x') else columna for columna in datos_nuevos.columns]

    #Concatenamos los nuevos datos con los antiguos de cada portal y los guardamos
    df_completo = pd.concat([datos_antiguos, datos_nuevos], ignore_index = True)

    df_completo.to_csv(ruta_datos + f'datos_{portal}.csv', index = False)

    #Devolvemos los nuevos datos para aplicarles las funciones de limpieza
    datos_nuevos.to_csv(ruta_datos + f'datos_nuevos_{portal}_{ultima_fecha}.csv', index = False)

# 1. Infoempleo Scraper

## 1.1. URL ofertas de empleo

In [27]:
def url_scraper_infoempleo(driver, dic_datos):
    nuevas = dic_datos['opcion']

    # Acepto cookies
    driver.find_element(By.ID, "onetrust-accept-btn-handler").click()
    sleep(1)
    
    url_empleos = []
    while True:

        try:
            driver.find_element(By.ID, "lightbox").find_element(By.CLASS_NAME, "close").click()
            sleep(1)
        except:
            pass
        
        if nuevas:
            # Boton ofertas en las últimos 15 dias:
            driver.find_element(By.ID, nuevas).click()
            nuevas = False
            sleep(1)

        soup = BeautifulSoup(driver.page_source, "html.parser")
        empleos = soup.find("div", class_= "main-content").find_all("li", class_= "offerblock")

        for url in empleos:
            try:
                url_oferta = "https://www.infoempleo.com" + url.find("a")["href"]
                url_empleos.append(url_oferta)
            except:
                pass

        try:
            # Hago scroll hasta el final:
            elemento_objetivo = driver.find_elements(By.CLASS_NAME, "related-offer-item")[-1]
            driver.execute_script("arguments[0].scrollIntoView(true);", elemento_objetivo)
            sleep(1)
            # Siguiente página:
            driver.find_element(By.CLASS_NAME, "pagination").find_element(By.CLASS_NAME, "next").click()
            sleep(2)
        except:
            break
            
    return url_empleos

## 1.2. Información ofertas de empleo

In [11]:
def scraper_infoempleo(driver, fecha_scrapeo, portal, nombres_columnas, ruta_datos, dic_datos):
    url_empleos = url_scraper_infoempleo(driver = driver, dic_datos = dic_datos)
    
    data = []
    contador = 0
    for empleo in url_empleos:
    
        driver.get(empleo)
        sleep(2)

        try:
            # Acepto cookies
            driver.find_element(By.ID, "onetrust-accept-btn-handler").click()
        except:
            pass

        # Cierro popup si aparece:
        try:
            driver.find_element(By.ID, "lightbox").find_element(By.CLASS_NAME, "close").click()
            sleep(1)
        except:
            pass

        soup = BeautifulSoup(driver.page_source, "html.parser")
        
        titulo = extraccion_datos_try_except_simple(soup, [("div", {'class' : "title-wrapper"}), ('h1',{})])

        empresa = extraccion_datos_try_except_simple(soup, [("div", {'class' : "title-wrapper"}), ('li',{'class' : 'companyname'})])

        presencialidad = extraccion_datos_try_except_simple(soup, [("div", {'class' : "title-wrapper"}), ('li',{'class' : 'badge'})])

        fecha = extraccion_datos_try_except_simple(soup, [("div", {'class' : "title-wrapper"}), ('li',{'class' : 'mt10'})])

        ubicacion = extraccion_datos_try_except_simple(soup, [("div", {'class' : "title-wrapper"}), ('li',{'class' : 'block'})])

        # Bloque de características:

        try:
            driver.find_element(By.CLASS_NAME, "areapos-vmore").click()
        except:
            pass

        bullet_points = soup.find("div", class_= "offer-excerpt").find_all("ul", class_= "inline")

        try:
            experiencia = bullet_points[0].find_all("p")[0].text
        except:
            experiencia = np.nan

        try:
            salario = bullet_points[0].find_all("p")[1].text
        except:
            salario = np.nan

        try:
            funciones = [funcion.text for funcion in bullet_points[1].find_all("li")[1:]]
        except:
            funciones = np.nan

        try:
            solicitudes = bullet_points[2].find_all("p")[1].text
        except:
            solicitudes = np.nan        

        try:
            tipo_contrato = bullet_points[3].find_all("p")[0].text
        except:
            tipo_contrato = np.nan

        try:
            jornada = bullet_points[3].find_all("p")[1].text
        except:
            jornada = np.nan  

        cuerpo_oferta = soup.find("div", class_= "offer").find_all("pre")

        try:
            descripcion = cuerpo_oferta[0].text.replace("\n", "").replace("*", "").strip()
        except:
            descripcion = np.nan    

        try:
            herramientas = cuerpo_oferta[1].text.replace("\n", "").replace("*", "").strip()
        except:
            herramientas = np.nan

        data.append([titulo, empresa, fecha, herramientas, descripcion, ubicacion, presencialidad, funciones, jornada, experiencia, tipo_contrato, salario, solicitudes, fecha_scrapeo, empleo, portal])
        contador += 1

        if contador % 100 == 0:
            sleep(20)
            
    guardar_datos(portal = portal, data = data, nombres_columnas = nombres_columnas, ruta_datos = ruta_datos)
    print(f'{portal} - Scrapeo completado')

# 2. Talenthacker Scraper

## 2.1. URL ofertas de empleo

In [12]:
def url_scraper_talenthacker(driver, dic_datos = None):

    # Acepto cookies:
    driver.find_element(By.CLASS_NAME, "q-banner__actions").find_element(By.CLASS_NAME, "buttons-wrapper").find_element(By.CLASS_NAME, "q-btn--standard").click()

    # Hago scroll hasta poder ver todas las ofertas disponibles:
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    while True:
        # Scroll:
        driver.execute_script("window.scrollTo(0,document.documentElement.scrollHeight);")
        sleep(3)
        # Calcula la altura del nuevo scroll y la compara con la del último:
        new_height = driver.execute_script("return document.documentElement.scrollHeight")

        if new_height == last_height:
            print("Ya no hay más página.")
            break
        last_height = new_height

    soup = BeautifulSoup(driver.page_source, "html.parser")

    talent_hacker = "https://talenthackers.net"
    ofertas = soup.find("div", class_= "jobs-list-wrapper").find_all("div", class_= "col-12")

    url_ofertas = []
    for oferta in ofertas:
        url_oferta = talent_hacker + oferta.find("a")["href"]
        url_ofertas.append(url_oferta)
        
    return url_ofertas

## 2.2. Información ofertas de empleo

In [13]:
def scraper_talenthacker(driver, fecha_scrapeo, portal, nombres_columnas, ruta_datos, dic_datos):
    url_empleos = url_scraper_talenthacker(driver = driver, dic_datos = dic_datos)
    
    bullet_point = dic_datos['bullet_point']
    
    data = []
    contador = 0
    for empleo in url_empleos:
        
        driver.get(empleo)
        sleep(2)
        
        try:
            # Acepto cookies
            driver.find_element(By.CLASS_NAME, "q-banner__actions").find_element(By.CLASS_NAME, "buttons-wrapper").find_element(By.CLASS_NAME, "q-btn--standard").click()
        except:
            pass
        
        soup = BeautifulSoup(driver.page_source, "html.parser")
        
        #########################  INFORMACION  ##########################################################################
        
        try:
            titulo = soup.find("span", class_= "spot-title").text
        except:
            titulo = np.nan
            
        try:
            herramientas = [herramienta.text for herramienta in soup.find("div", class_= "spot-skills").find_all("a")]
        except:
            herramientas = np.nan
            
        try:
            descripcion = soup.find_all("div", class_= "block full-width text-body2")[0].text
        except:
            descripcion = np.nan
            
        try:
            funciones = soup.find_all("div", class_= "block full-width text-body2")[1].text
        except:
            funciones = np.nan
            
            
        imagenes = soup.find_all("div", class_= "th-body-md flex no-wrap q-ma-sm")
        for imagen in imagenes:
            img = imagen.find("img")["src"]
            
            if img == bullet_point["ubicacion"]:
                try:
                    ubicacion = imagen.text
                except:
                    ubicacion = np.nan
                    
            elif img == bullet_point["presencialidad"]:
                try:
                    presencialidad = imagen.text.strip()
                except:
                    presencialidad = np.nan
                    
            elif img == bullet_point["jornada_tipo"]:
                try:
                    tipo_contrato = imagen.text.split("·")[0].strip()
                    jornada = imagen.text.split("·")[1].strip()
                except:
                    tipo_contrato = np.nan 
                    jornada = np.nan
            
            if img == bullet_point["experiencia"]:
                try:
                    experiencia = imagen.text.strip()
                except:
                    experiencia = np.nan

            
            elif img == bullet_point["salario"]:
                try:
                    salario = imagen.text.strip()
                except:
                    salario = np.nan

        
        ###################################################################################################################
        data.append([titulo, herramientas, descripcion, ubicacion, presencialidad, funciones, jornada, experiencia, tipo_contrato, salario, fecha_scrapeo, empleo, portal])
        contador += 1

        if contador % 100 == 0:
            sleep(20)
            
            
    guardar_datos(portal = portal, data = data, nombres_columnas = nombres_columnas, ruta_datos = ruta_datos)
    print(f'{portal} - Scrapeo completado')

# 3. Tecnoempleo Scraper

## 3.1. Número de páginas

In [14]:
def pag_counter(driver, dic_datos):
    limite = dic_datos['limite']
    nuevas = dic_datos['nuevas']
    
    # Entro en tecnoempleo:
    if nuevas:
        url_paginas = f"https://www.tecnoempleo.com/ofertas-trabajo/?pagina=50"
        
    else:
        url_paginas = f"https://www.tecnoempleo.com/ofertas-trabajo/?pagina={limite}"
    
    driver.get(url_paginas)
    
    # Acepto cookies
    driver.find_elements(By.CLASS_NAME, "col-6")[0].click()
    
    # Saco el número de páginas
    soup = BeautifulSoup(driver.page_source, "html.parser")
    num_paginas = int(soup.find("li", class_= "active").text)
    
    return num_paginas    

## 3.2. Información ofertas de empleo

In [15]:
def scraper_tecnoempleo(driver, fecha_scrapeo, portal, nombres_columnas, ruta_datos, dic_datos):
    num_paginas = pag_counter(driver, dic_datos)
    
    data = []
    
    contador = 0
    for pagina in range(num_paginas + 1):
        
        tecno_url = f"https://www.tecnoempleo.com/ofertas-trabajo/?pagina={pagina}"

        driver.get(tecno_url)
        sleep(1)
        
        # Acepto cookies
        try:
            driver.find_elements(By.CLASS_NAME, "col-6")[0].click()
        except:
            pass

        # Saco las urls de cada puesto ofertado y saco la información
        soup_page = BeautifulSoup(driver.page_source, "html.parser")

        ofertas_empleo = soup_page.find_all("div", class_= "col-10")

        for oferta in ofertas_empleo:

            # URL oferta de empleo
            url_oferta = oferta.find("a")["href"]
            # Entro en la oferta
            driver.get(url_oferta)
            sleep(1)
            # Saco información relevante de la oferta
            soup = BeautifulSoup(driver.page_source, "html.parser")

            lista_li = soup.find("ul", class_= "fs--15").find_all("li", class_= "border-bottom")

            ubicacion, funciones, jornada, experiencia, tipo_contrato, salario = [np.nan] * 6
            for li in lista_li:

                tag = li.find("span", class_= "d-inline-block").text

                if tag == "Ubicación" : 
                    ubicacion = li.find("span", class_= "float-end").text.strip().replace("\xa0", "").replace("\t", "").replace("\n", "")

                elif tag == "Funciones":
                    funciones = li.find("span", class_= "float-end").text.strip().replace("\xa0", "").replace("\t", "").replace("\n", "")

                elif tag == "Jornada":
                    jornada = li.find("span", class_= "float-end").text.strip().replace("\xa0", "").replace("\t", "").replace("\n", "")

                elif tag == "Experiencia":
                    experiencia = li.find("span", class_= "float-end").text.strip().replace("\xa0", "").replace("\t", "").replace("\n", "")

                elif tag == "Tipo contrato":
                    tipo_contrato = li.find("span", class_= "float-end").text.strip().replace("\xa0", "").replace("\t", "").replace("\n", "")

                elif tag == "Salario":
                    salario = li.find("span", class_= "float-end").text.strip().replace("\xa0", "").replace("\t", "").replace("\n", "")

            titulo = soup.find("div", class_= "col-lg-8").find("h1").text.replace("\t", "").replace("Urgente\n", "").strip()
        
            try:
                empresa = soup.find("div", class_= "col-lg-8").find("a", class_= "fs--18").text.strip()
            except:
                empresa = np.nan

            fecha = soup.find("div", class_= "col-lg-8").find("span", "ml-4").text.strip().replace("Actualizada", "")

            herramientas = soup.find("ul", class_= "fs--15").find("li", class_= "mb-3").text.strip().replace("\n", ", ")

            descripcion = soup.find("div", class_= "mt-4").text.replace("\n", " ").replace("\t", " ").replace("Descripción de la oferta de empleo", "").strip()

            data.append([titulo, empresa, fecha, herramientas, descripcion, ubicacion, funciones, jornada, experiencia, tipo_contrato, salario, fecha_scrapeo, url_oferta, portal])
            
        contador =+ 1
        if contador % 2 == 0:
            sleep(3)
            
    guardar_datos(portal = portal, data = data, nombres_columnas = nombres_columnas, ruta_datos = ruta_datos)
    sleep(2)

# 4. Infojobs Scraper

## 4.1. URL ofertas de empleo

In [16]:
def url_scraper_infojobs(driver, dic_datos):
    opcion = dic_datos['opcion']
    api_key = dic_datos["api_key"]
    
    pagina = 1
    
    url_infojobs = f"https://www.infojobs.net/jobsearch/search-results/list.xhtml?keyword=&categoryIds=150&segmentId=&page={pagina}&sortBy=PUBLICATION_DATE&onlyForeignCountry=false&sinceDate={opcion}"
    
    # Initialize:
    driver.get(url_infojobs)
    sleep(4)
    driver.get(url_infojobs)
    sleep(4)
    
    ##################################################Captcha##################################################

    driver.find_element(By.CLASS_NAME, "geetest_radar_tip").click()
    sleep(5)
    driver.find_element(By.CLASS_NAME, "geetest_voice").click()
    sleep(5)
    driver.find_element(By.CLASS_NAME, "geetest_replay").click()
    audio_data = record_audio()
    sleep(2)
    save_audio(audio_data)
    sleep(2)

    client = OpenAI(api_key= api_key)
    audio_file = os.getcwd() + "\\recorded_audio.mp3"

    audio_file= open(audio_file, "rb")
    transcript = client.audio.transcriptions.create(model="whisper-1", file=audio_file)

    codigo = transcript.text.replace("Introduzca lo que oiga.", "").replace(", ", "").replace(".", "").strip()

    barra = driver.find_element(By.CLASS_NAME, "geetest_input")
    barra.send_keys(codigo)
    sleep(2)
    driver.find_element(By.CLASS_NAME, "geetest_box").find_element(By.CLASS_NAME, "geetest_btn").click()
    sleep(5)
    ###########################################################################################################

    # Acepto cookies
    driver.find_elements(By.ID, "didomi-notice-agree-button")[0].click()
    sleep(1)
    
    url_empleos = []
    while True:

        # Saco url empleos:
        soup_urls = BeautifulSoup(driver.page_source, "html.parser")

        try:
            urls = soup_urls.find_all("div", class_= "ij-OfferCardContent-description")
        except:
            break

        for url in urls:
            oferta_url = url.find("h2").find("a")["href"].replace("//", "")
            url_empleos.append(oferta_url)

        pagina += 1
        driver.find_elements(By.CLASS_NAME, "sui-MoleculePagination-item")[-1].click()
        sleep(2)

        lim_paginas = len(driver.find_elements(By.CLASS_NAME, "sui-MoleculePagination-item"))

        if pagina > 1:
            if lim_paginas < 7:
                break
                
    url_empleos_final = ["https://" + empleo for empleo in url_empleos]
    
    return url_empleos_final

## 4.2. Información ofertas de empleo

In [17]:
def scraper_infojobs(driver, fecha_scrapeo, portal, nombres_columnas, ruta_datos, dic_datos):
    url_empleos = url_scraper_infojobs(driver = driver, dic_datos = dic_datos)
    api_key = dic_datos['api_key']
    
    data = []
    contador = 0
    
    for empleo in url_empleos:

        driver.get(empleo)
        sleep(4)
        
        try:
            
            ##########Captcha##########
            driver.find_element(By.CLASS_NAME, "geetest_radar_tip").click()
            sleep(2)
            driver.find_element(By.CLASS_NAME, "geetest_voice").click()
            sleep(2)
            driver.find_element(By.CLASS_NAME, "geetest_replay").click()
            audio_data = record_audio(duration=15)
            sleep(2)
            save_audio(audio_data)

            client = OpenAI(api_key= api_key)
            audio_file = os.getcwd() + "\\recorded_audio.mp3"

            audio_file= open(audio_file, "rb")
            transcript = client.audio.transcriptions.create(model="whisper-1", file=audio_file)

            codigo = transcript.text.replace("Introduzca lo que oiga.", "").replace(", ", "").replace(".", "").strip()

            barra = driver.find_element(By.CLASS_NAME, "geetest_input")
            barra.send_keys(codigo)
            sleep(2)
            driver.find_element(By.CLASS_NAME, "geetest_box").find_element(By.CLASS_NAME, "geetest_btn").click()
            sleep(5)
            ###########################

            # Acepto cookies
            driver.find_elements(By.ID, "didomi-notice-agree-button")[0].click()
            sleep(1)

        except:
            next
            
        # Saco información relevante de la oferta:    
        soup = BeautifulSoup(driver.page_source, "html.parser")

        try:
            captcha = soup.find("div", attrs= attrs_ad).text        
            if captcha == "Hacer clic para comprobarReintentar":
                ##########Captcha##########
                driver.find_element(By.CLASS_NAME, "geetest_radar_tip").click()
                sleep(2)
                driver.find_element(By.CLASS_NAME, "geetest_voice").click()
                sleep(2)
                driver.find_element(By.CLASS_NAME, "geetest_replay").click()
                audio_data = record_audio(duration=15)
                sleep(2)
                save_audio(audio_data)

                client = OpenAI(api_key= api_key)
                audio_file = os.getcwd() + "\\recorded_audio.mp3"

                audio_file= open(audio_file, "rb")
                transcript = client.audio.transcriptions.create(model="whisper-1", file=audio_file)

                codigo = transcript.text.replace("Introduzca lo que oiga.", "").replace(", ", "").replace(".", "").strip()

                barra = driver.find_element(By.CLASS_NAME, "geetest_input")
                barra.send_keys(codigo)
                sleep(2)
                driver.find_element(By.CLASS_NAME, "geetest_box").find_element(By.CLASS_NAME, "geetest_btn").click()
                sleep(5)
                ###########################
                
        except:
            pass
        
        titulo = extraccion_datos_try_except_simple(soup, [('div', {'class' : 'heading-addons'}), ('h1', {"id": "prefijoPuesto"})])

        empresa = extraccion_datos_try_except_simple(soup, [('div', {'class' : 'heading-addons'}), ('a', {"class": "link"})])

        ubicacion = extraccion_datos_try_except_simple(soup, [('div', {'class' : 'row-matrioska'}), ('span', {"id": "prefijoPoblacion"})]).replace(",", "")

        descripcion = extraccion_datos_try_except_simple(soup, [('div', {"id": "prefijoDescripcion1"})])

        herramientas = extraccion_datos_try_except_compleja(soup, [("div", {'class' : "inner-expanded"}), ("ul", {'class' : "list-default"}), ('a', {})], elementos_a_encontrar = 'lista_completa')

        funciones = extraccion_datos_try_except_simple(soup, [("div", {'class' : "border-top"}), ("ul", {}), ('span', {'class' : "list-default-text"})])
        
        try:
            fecha = soup.find("div", class_= "row-matrioska").find("span", class_= "marked").text
        except:
            try:
                lista_bullets = soup.find("div", class_= "row-matrioska").find("ul", class_= "list-bullet-default").find_all("li")
                for bullet in lista_bullets:
                    tag_bullet = bullet.text.strip()[:9]
                    if tag_bullet == "Publicada":
                        fecha = bullet.text.strip()[12:]
            except:
                fecha = np.nan

        attrs_ubic= {"id": "prefijoPoblacion"}
       
        try:
            lista_li = soup.find("div", class_= "row-matrioska").find_all("li")        

            for li in lista_li:

                tag = li.find("span").text[:7]

                if tag == "Salario":
                    salario = li.find("span").text[8:]

                elif tag == "Experie":
                    experiencia = li.find("span").text[20:]

                elif tag == "Tipo de":

                    tipo_contrato = li.find("span").text[18:].split(",")[0]

                    jornada = li.find("span").text[18:].split(",")[-1]
        except:
            salario = np.nan
            experiencia = np.nan
            tipo_contrato = np.nan
            jornada = np.nan

        data.append([titulo, empresa, fecha, herramientas, descripcion, ubicacion, funciones, jornada, experiencia, tipo_contrato, salario, fecha_scrapeo, empleo, portal])
        contador += 1

        if contador % 100 == 0:
            sleep(20)
            
    df_infojobs = pd.DataFrame(data)
    df_infojobs = df_infojobs.dropna(how= "all")
    data = df_infojobs.values
    guardar_datos(portal = portal, data = data, nombres_columnas = nombres_columnas, ruta_datos = ruta_datos)
    print(f'{portal} - Scrapeo completado')

In [18]:
df_prueba = pd.read_csv('C:\\Users\\regue\\Desktop\\Data Science Projects\\PROJECTS\\IT_Job_Spain_Project\\Datos\\datos_sin_procesar\\' + "datos_infojobs.csv")

In [19]:
df_prueba.values

array([['Full Stack Developer',
        'Tokio Marine Europe S.A Sucursal en España  Products',
        'hace 1h', ..., '2024-01-26', 'infojobs', nan],
       ['Operador Experto en Sistema Cloud AWS', 'EZENTIS', 'hace 1h',
        ..., '2024-01-26', 'infojobs', nan],
       ['ARQUITECTO/A SOFTWARE TECNOLOGÍA FRONT', 'CASER - Corporativas',
        'hace 1h', ..., '2024-01-26', 'infojobs', nan],
       ...,
       ['Técnicos/as de Soporte IT - Titulados/as en Grado Superior Informática/Telecomunicaciones',
        'Grupo Zelenza', 'ace 1d', ..., '2024-02-10', 'infojobs',
        'https://www.infojobs.net/madrid/tecnicos-soporte-it-titulados-grado-superior-informatica-telecomunicaciones/of-ia259d0ed6a44fbbceb36f724237f16?applicationOrigin=search-new&page=21&sortBy=PUBLICATION_DATE'],
       ['Un/a Programador/a ERP Axional Deister', 'Clubs de Fitness DiR',
        'ace 1d', ..., '2024-02-10', 'infojobs',
        'https://www.infojobs.net/barcelona/un-programador-erp-axional-deister/of-i6

# 5. Ticjob Scraper

## 5.1. Búsqueda ofertas de empleo

In [20]:
def busqueda_ticjob(driver, dic_datos = None):
    # Ordenamos los resultados por fecha
    driver.find_element(By.CLASS_NAME, 'sort-by-date-container').click()

    sleep(5)

## 5.2 Información ofertas de empleo

In [21]:
def scraper_ticjob(driver, fecha_scrapeo, portal, nombres_columnas, ruta_datos, dic_datos):
    intervalo_temporal_busqueda = dic_datos['intervalo_temporal_busqueda']
    intervalo_temporal_busqueda = timedelta(days = intervalo_temporal_busqueda)
    
    # Ordenamos los resultados por fecha
    driver.find_element(By.CLASS_NAME, 'sort-by-date-container').click()

    sleep(5)
    
    data = []
    final_scrapeo = False
    
    while True:
        
        ofertas = driver.find_elements(By.CLASS_NAME, 'job-card')

        for oferta in ofertas:
            #Extraemos los datos con selenium, ya que al obtener la url de la pagina nos da los de una busqueda generica sin nuetros criterios
            #Extraemos solo los datos del intervalo temporal seleccionado
            fecha = oferta.find_element(By.CSS_SELECTOR, 'div[class = "job-card-label date-field"]').text
            fecha = datetime.strptime(fecha, '%d/%m/%Y').date()

            if fecha < fecha_scrapeo - intervalo_temporal_busqueda:
                final_scrapeo = True
                break

            url = oferta.find_element(By.TAG_NAME, 'a').get_attribute('href')

            response = requests.get(url)

            #Extraemos los datos con soup
            soup = BeautifulSoup(response.text, "html.parser")

            titulo = extraccion_datos_try_except(soup, 'h1', {'id' : 'job-title'})

            empresa = soup.find('a', class_ = 'company-image')['title']

            descripcion = extraccion_datos_try_except(soup, 'div', {'class' : 'job-offer job-offer-content'})

            ubicacion = extraccion_datos_try_except(soup, 'li', {'class' : 'multi-job-location-apply'})
            if (ubicacion != np.nan) and (';' in ubicacion):
                ubicacion = ubicacion.split(';')

            experiencia = extraccion_datos_try_except(soup, 'li', {'id' : 'summaryExp'})

            localizacion = soup.find('li', class_ = 'multi-job-location-apply')
            tipo_contrato = localizacion.find_next('li').get_text(strip = True)

            salario = extraccion_datos_try_except(soup, 'li', {'id' : 'summarySalary'})
            # Eliminamos los salarios == '0' ya que no son datos reales que se muestren en la pagina, ya que corresponden con campos vacios
            salario = salario if salario != '0' else np.nan

            herramientas = soup.find('div', class_ = 'search-criteria-tags').find_all('a')
            herramientas = [herramienta.text for herramienta in herramientas]

            data.append([titulo, empresa, fecha, herramientas, descripcion, ubicacion, experiencia, tipo_contrato, salario, fecha_scrapeo, url, portal])
            sleep(1)


        #Guardamos cada vez que termina de sacrapear una pagina
        guardar_datos(portal = portal, nombres_columnas = nombres_columnas, data = data, ruta_datos = ruta_datos)
        data = []

        paginas_totales = driver.find_element(By.CLASS_NAME, 'page-list').text.split('\n')[-1]
        paginas_totales = int(paginas_totales)
        pagina_actual = int(driver.find_element(By.CLASS_NAME, 'current').text)

        # Pasamos a la siguiente pagina hasta llegar a la ultima
        if (paginas_totales != pagina_actual) and not final_scrapeo:
            # Pasamos a la siguiente pagina
            siguiente_pagina = driver.find_element(By.CLASS_NAME, 'next')
            driver.execute_script("arguments[0].scrollIntoView();", siguiente_pagina)
            siguiente_pagina.click()
            sleep(2)
        else:
            print(f'{portal} - Scrapeo completado')
            break

# 6. Indeed Scraper

## 6.1. Búsqueda ofertas de empleo

In [22]:
def busqueda_indeed(driver, empleo, dic_datos):
    ubicacion_a_buscar = dic_datos['ubicacion_a_buscar']
    
    intervalo_temporal_busqueda = dic_datos['intervalo_temporal_busqueda']
    
    # Buscar empleo
    buscador_empleo = driver.find_element(By.ID, 'text-input-what')
    buscador_empleo.clear()
    sleep(1)
    buscador_empleo.send_keys(empleo)
    sleep(1)

    # Buscar ubicacion
    buscador_ubicacion = driver.find_element(By.ID, 'text-input-where')
    #buscador_ubicacion.send_keys(ubicacion_a_buscar)
    sleep(1)
    buscador_ubicacion.send_keys(Keys.ENTER)
    sleep(1)

    #Seleccionamos la fecha de publicacion
    driver.find_element(By.ID, 'filter-dateposted').click()
    sleep(2)
    fechas_publicacion = driver.find_element(By.ID, 'filter-dateposted-menu')
    opciones_fechas_publicacion = fechas_publicacion.text.split('\n')

    #Seleccionamos los elementos interactuables de las fechas de publicacion
    botones_fechas_publicacion = fechas_publicacion.find_elements(By.TAG_NAME, 'li')

    #Pulsamos el que tiene el intervalo de tiempo seleccionado
    [publicaciones_seleccionadas for publicaciones_seleccionadas in  botones_fechas_publicacion if intervalo_temporal_busqueda in publicaciones_seleccionadas.text][0].click()

## 6.2 Información ofertas de empleo

In [23]:
def scraper_indeed(empleo, driver, fecha_scrapeo, portal, nombres_columnas, ruta_datos, dic_datos):
    tipos_trabajos = dic_datos['tipos_trabajos']
    
    while True:
        data = []
        sleep(4)
        try:
            #Cerrar banner
            banner = driver.find_element(By.CSS_SELECTOR, 'button[aria-label = "cerrar"]')
            banner.click()

        except:
            next

        ofertas = driver.find_elements(By.CLASS_NAME, 'job_seen_beacon')
        for oferta in ofertas:
            driver.execute_script("arguments[0].scrollIntoView();", oferta)
            sleep(2)
            oferta.click()
            sleep(3)

            titulo = WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'h2[data-testid = "jobsearch-JobInfoHeader-title"]')))
            titulo = titulo.text.split('\n')[0]

            empresa = WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div[data-testid = "inlineHeader-companyName"]')))
            empresa = empresa.text

            descripcion = WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.ID, 'jobDescriptionText')))
            descripcion = descripcion.text.replace('\n', ' ')

            fecha = WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'span[data-testid = "myJobsStateDate"]')))
            fecha = fecha.text.split('\n')[1]

            ubicacion_presencialidad = WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div[data-testid = "inlineHeader-companyLocation"]')))
            ubicacion_presencialidad = ubicacion_presencialidad.text.split('•')

            if len(ubicacion_presencialidad) > 1:
                ubicacion, presencialidad = ubicacion_presencialidad
            else:
                if ubicacion_presencialidad in tipos_trabajos:
                    presencialidad = ubicacion_presencialidad[0]
                    ubicacion = np.nan
                else:
                    ubicacion = ubicacion_presencialidad[0]
                    presencialidad = np.nan
            try:
                beneficios = driver.find_element(By.ID, 'benefits').text
                beneficios = beneficios.replace('Beneficios\nObtenidos de la descripción completa del empleo\n','').split('\n')

            except:
                beneficios = np.nan
                next

            detalles_a_buscar = {'Salario' : np.nan, 'Tipo de empleo' : np.nan}   

            try:
                detalles = driver.find_element(By.ID, 'jobDetailsSection')
                detalles = detalles.text.split('\n')    

                for detalle in detalles_a_buscar.keys():
                    if detalle in detalles:
                        idx = detalles.index(detalle)
                        detalles_a_buscar[detalle] = detalles[idx + 1]
            except:
                next

            salario, jornada = detalles_a_buscar.values()

            url = driver.current_url

            data.append([titulo, empresa, fecha, descripcion, ubicacion, jornada, presencialidad, salario, beneficios, fecha_scrapeo, url, portal])
            
        #Guardamos cada vez que termina de sacrapear una pagina
        guardar_datos(portal = portal, nombres_columnas = nombres_columnas, data = data, ruta_datos = ruta_datos)

        pagina_actual = int(driver.find_element(By.CSS_SELECTOR, 'a[data-testid = "pagination-page-current"]').text)

        try:
            driver.find_element(By.CSS_SELECTOR, f'a[data-testid = "pagination-page-{pagina_actual + 1}"]').click()
            sleep(3)
        except:
            print(f'{portal} - Scrapeo completado para {empleo}')
            break

# 7. Linkedin Scraper

## 7.1. Búsqueda ofertas de empleo

In [24]:
def busqueda_linkedin(driver, empleo, dic_datos):    
    usuario = dic_datos['usuario']
    password = dic_datos['password']
    intervalo_temporal_busqueda = dic_datos['intervalo_temporal_busqueda']
    try:
        #Iniciamos sesion
        driver.find_element(By.ID, value = 'session_key').send_keys(usuario)
        sleep(1)
        driver.find_element(By.ID, value = 'session_password').send_keys(password)
        sleep(1)
        driver.find_element(By.CSS_SELECTOR, value = 'button[data-id = "sign-in-form__submit-btn"').click()
        sleep(2)
    
    except:
        next
        
    #Accedemos a los empleos
    empleos = WebDriverWait(driver, 120).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'span[title = "Empleos"]')))
    empleos.click()
    sleep(2)

    #Buscamos empleo
    buscador = driver.find_element(By.CSS_SELECTOR, value = 'input[class = "jobs-search-box__text-input jobs-search-box__keyboard-text-input"]')
    buscador.send_keys(empleo)
    sleep(1)
    buscador.send_keys(Keys.ENTER)
    sleep(2)

    #Pulsamos todos los filtros
    todos_los_filtros = WebDriverWait(driver, 120).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div[class = "relative mr2"]')))
    todos_los_filtros.click()

    # Localizamos las fechas de publicacion de los empleos
    fechas_publicacion = [filtro for filtro in driver.find_elements(By.CLASS_NAME, value = 'search-reusables__secondary-filters-filter') if 'Fecha de publicación' in filtro.text][0]
    opciones_fechas_publicacion = fechas_publicacion.text.split('\n')
    opciones_fechas_publicacion = [opcion_fechas_publicacion for opcion_fechas_publicacion in opciones_fechas_publicacion if '«' not in opcion_fechas_publicacion][1:]

    # Obtener el valor seleccionado por el usuario
    '''publicaciones_seleccionadas = [publicaciones_seleccionadas for publicaciones_seleccionadas in fechas_publicacion.find_elements(By.TAG_NAME, 'li') if intervalo_temporal_busqueda in publicaciones_seleccionadas.text][0]
    publicaciones_seleccionadas.find_element(By.TAG_NAME, 'label').click()
    '''
    # Extraemos los tipos de empleos disponibles para la busqueda
    tipos_empleos = [filtro.text for filtro in driver.find_elements(By.CLASS_NAME, value = 'search-reusables__secondary-filters-filter') if 'Tipo de empleo' in filtro.text][0].split('\n')
    tipos_empleos = [tipo for tipo in tipos_empleos if '«' not in tipo][1:]

    # Extraemos los tipos_presencialidad disponibles para la busqueda
    tipos_presencialidad = [filtro.text for filtro in driver.find_elements(By.CLASS_NAME, value = 'search-reusables__secondary-filters-filter') if 'En remoto' in filtro.text][0].split('\n')
    tipos_presencialidad = [presencialidad for presencialidad in tipos_presencialidad if '«' not in presencialidad][1:]

    sleep(1)
    
    #Cerramos los filtros
    cerrar_filtros = [elemento for elemento in driver.find_elements(By.CSS_SELECTOR, 'span[class = "a11y-text"]') if 'Todos los filtros' in elemento.text][0]
    cerrar_filtros.find_element(By.XPATH, ".//following-sibling::button").click()
    sleep(1)
     
    # Seleccion fechas publicacion
    fechas_publicacion = [filtro for filtro in driver.find_elements(By.CLASS_NAME, 'search-reusables__primary-filter') if 'Fecha' in filtro.text][0]
    fechas_publicacion.click()

    publicaciones_seleccionadas = [filtro for filtro in fechas_publicacion.find_elements(By.CLASS_NAME, 'search-reusables__value-label') if intervalo_temporal_busqueda in filtro.text][0]
    publicaciones_seleccionadas.click()

    botones = fechas_publicacion.find_element(By.XPATH, ".//following-sibling::div")

    botones.find_elements(By.TAG_NAME, 'button')[-1].click()
    sleep(3)

## 7.2 Información ofertas de empleo

In [25]:
def scraper_linkedin(empleo, driver, fecha_scrapeo, portal, nombres_columnas, ruta_datos, dic_datos):

    while True:
        data = []
        sleep(4)

        # Obtenemos todas las ofertas de la pagina actual
        while True:
            ofertas_visibles = driver.find_elements(By.CSS_SELECTOR, value = 'div[data-view-name = "job-card"]')
            if not ofertas_visibles:
                print(f'{portal} - Scrapeo completado para {empleo}')
                break

            driver.execute_script("arguments[0].scrollIntoView();", ofertas_visibles[-1])
            sleep(1)
            nuevas_ofertas_visibles = driver.find_elements(By.CSS_SELECTOR, value = 'div[data-view-name = "job-card"]')
            if len(nuevas_ofertas_visibles) != len(ofertas_visibles):
                next
            else:
                break

        #Extraccion de datos de cada oferta
        for oferta in ofertas_visibles:
            driver.execute_script("arguments[0].scrollIntoView();", oferta)
            sleep(2)
            oferta.click()
            sleep(2)
            datos = driver.find_element(By.CSS_SELECTOR, value = 'div[class = "job-details-jobs-unified-top-card__primary-description-without-tagline mb2"]').text
            sleep(2)

            # try except para no perder datos, ya que si no se consigue realizar la accion se volvera a la oferta mas tarde
            try:
                titulo, empresa = oferta.text.split('\n')[:2]
            except:
                ofertas_visibles.extend([oferta])
                continue

            try:
                ubicacion, fecha, solicitudes = datos.split('·')[1:]
            except:
                ubicacion, fecha, solicitudes = np.nan, np.nan, np.nan

            descripcion = driver.find_element(By.ID, value = 'job-details').text.replace('\n', '')

            jornada_presencialidad = driver.find_elements(By.CLASS_NAME, value = 'job-details-jobs-unified-top-card__job-insight')[0].text
            try:
                jornada = [jornada for jornada in tipos_empleos if jornada in jornada_presencialidad][0]
            except:
                jornada = np.nan
                next
            try:
                presencialidad = [presencialidad for presencialidad in tipos_presencialidad if presencialidad in jornada_presencialidad][0]
            except:
                presencialidad = np.nan
                next
            #mostrar todas las aptitudes
            while True:
                try:
                    #Tratamos de buscar si hay aptitudes
                    try:
                        [enlace_aptitudes for enlace_aptitudes in driver.find_elements(By.CLASS_NAME, value = 'app-aware-link') if 'Aptitudes' in enlace_aptitudes.text][0].click()
                        sleep(2)

                        [mostrar_aptitudes for mostrar_aptitudes in driver.find_elements(By.CLASS_NAME, value = 'artdeco-button__text') if mostrar_aptitudes.text == 'Mostrar todas las aptitudes'][0].click()
                        sleep(2)

                        try:
                            caja_aptitudes = driver.find_element(By.CLASS_NAME, value = 'job-details-skill-match-status-list')

                        except NoSuchElementException:
                            #Salir de la caja de aptitudes
                            [boton for boton in driver.find_elements(By.CLASS_NAME, value = 'artdeco-button__text') if boton.text == 'Done'][0].click()
                            continue

                        sleep(4)
                        herramientas = [aptitud.text.replace('\nAñadir', '') for aptitud in caja_aptitudes.find_elements(By.TAG_NAME, value = 'li')]

                        sleep(2)

                        #Salir de la caja de aptitudes
                        [boton for boton in driver.find_elements(By.CLASS_NAME, value = 'artdeco-button__text') if boton.text == 'Done'][0].click()

                        break
                    except StaleElementReferenceException:
                        next
                except:

                    heramientas = np.nan
                    break

            url = driver.current_url

            data.append([titulo, empresa, fecha, herramientas, descripcion, ubicacion, jornada, presencialidad, solicitudes, fecha_scrapeo, url, portal])

        #Guardamos cada vez que termina de sacrapear una pagina
        guardar_datos(portal = portal, nombres_columnas = nombres_columnas, data = data, ruta_datos = ruta_datos)

        pagina_actual = driver.find_element(By.CSS_SELECTOR, 'li[class = "artdeco-pagination__indicator artdeco-pagination__indicator--number active selected ember-view"]')

        try:
            pagina_actual.find_element(By.XPATH, ".//following-sibling::li").click()

        except:
            print(f'{portal} - Scrapeo completado para {empleo}')
            break

# Datos JSON

In [26]:
diccionario_datos = {
    'portales_busqueda': {
        'ticjob': {
            'busqueda_por_empleo' : False,
            'busqueda_por_url' : False,
           
            'url': 'https://ticjob.es/esp/busqueda',
            'nombres_columnas': ["titulo", "empresa", "fecha", "herramientas", "descripcion", "ubicacion", "experiencia", "tipo_contrato", "salario", "fecha_scrapeo", "url", "portal"],
            
            'dic_datos' : {'intervalo_temporal_busqueda': 10,
                           'opciones_fechas_publicacion' : None}
        },
 
        'indeed': {
            'busqueda_por_empleo' : True,
            'busqueda_por_url' : False,
            
            'url': 'https://es.indeed.com/',
            'nombres_columnas': ['titulo', 'empresa', 'fecha', 'descripcion', 'ubicacion', 'jornada', 'presencialidad', 'salario', 'beneficios', 'fecha_scrapeo', 'url', 'portal'],
            
            'dic_datos' : {'ubicacion_a_buscar': 'España',
                           'intervalo_temporal_busqueda': 'Últimos 7 días',
                           'opciones_fechas_publicacion': ['Últimas 24 horas', 'Últimos 3 días', 'Últimos 7 días', 'Últimos 14 días'],
                           'tipos_trabajos' : ('Remoto híbrido', 'Teletrabajo')}
        },

        'linkedin': {
            'busqueda_por_empleo' : True,
            'busqueda_por_url' : False,
            
            'url': 'https://www.linkedin.com/',
            'nombres_columnas': ['titulo', 'empresa', 'fecha', 'herramientas', 'descripcion', 'ubicacion', 'jornada', 'presencialidad', 'solicitudes', 'fecha_scrapeo', 'url', 'portal'],
            
            'dic_datos' : {'usuario' : None,
                           'password' : None,
                           'intervalo_temporal_busqueda': 'Semana pasada',
                           'opciones_fechas_publicacion': ['Últimas 24 horas', 'Semana pasada']}
            
        },
  
        'infoempleo': {
            'busqueda_por_empleo' : False,
            'busqueda_por_url' : True,
            
            'url': 'https://www.infoempleo.com/trabajo/area-de-empresa_tecnologia-e-informatica/',
            'nombres_columnas': ["titulo", "empresa", "fecha", "herramientas", "descripcion", "ubicacion", "presencialidad", "funciones", "jornada", "experiencia", "tipo_contrato", "salario", "solicitudes", "fecha_scrapeo", "url", "portal"],
            
            'dic_datos' : {'opcion' : 'fechapublicacion3',
                           'opciones_fechas_publicacion' : ['fechapublicacion1', 'fechapublicacion2', 'fechapublicacion3']}
        },
     
        'talenthacker': {
            'busqueda_por_empleo' : False,
            'busqueda_por_url' : True,
            
            'url': 'https://talenthackers.net/spots/',
            'nombres_columnas': ["titulo", "herramientas", "descripcion", "ubicacion", "presencialidad", "funciones", "jornada", "experiencia", "tipo_contrato", "salario", "fecha_scrapeo", "url", "portal"],
            
            'dic_datos' : {'bullet_point' : {'ubicacion': 'data:image/svg+xml;base64,PHN2ZyB3aWR0aD0iMjQiIGhlaWdodD0iMjQiIHZpZXdCb3g9IjAgMCAyNCAyNCIgZmlsbD0ibm9uZSIgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIj4KPHBhdGggZD0iTTEyIDJDOC4xMyAyIDUgNS4xMyA1IDlDNSAxNC4yNSAxMiAyMiAxMiAyMkMxMiAyMiAxOSAxNC4yNSAxOSA5QzE5IDUuMTMgMTUuODcgMiAxMiAyWk03IDlDNyA2LjI0IDkuMjQgNCAxMiA0QzE0Ljc2IDQgMTcgNi4yNCAxNyA5QzE3IDExLjg4IDE0LjEyIDE2LjE5IDEyIDE4Ljg4QzkuOTIgMTYuMjEgNyAxMS44NSA3IDlaIiBmaWxsPSIjNjI3QTc4Ii8+CjxwYXRoIGQ9Ik0xMiAxMS41QzEzLjM4MDcgMTEuNSAxNC41IDEwLjM4MDcgMTQuNSA5QzE0LjUgNy42MTkyOSAxMy4zODA3IDYuNSAxMiA2LjVDMTAuNjE5MyA2LjUgOS41IDcuNjE5MjkgOS41IDlDOS41IDEwLjM4MDcgMTAuNjE5MyAxMS41IDEyIDExLjVaIiBmaWxsPSIjNjI3QTc4Ii8+Cjwvc3ZnPgo=',
                                             'presencialidad': 'data:image/svg+xml;base64,PHN2ZyB3aWR0aD0iMjQiIGhlaWdodD0iMjQiIHZpZXdCb3g9IjAgMCAyNCAyNCIgZmlsbD0ibm9uZSIgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIj4KPHBhdGggZD0iTTE3IDE1SDE5VjE3SDE3VjE1Wk0xNyAxMUgxOVYxM0gxN1YxMVpNMTcgN0gxOVY5SDE3VjdaTTEzLjc0IDdMMTUgNy44NFY3SDEzLjc0WiIgZmlsbD0iIzYyN0E3OCIvPgo8cGF0aCBkPSJNMTAgM1Y0LjUxTDEyIDUuODRWNUgyMVYxOUgxN1YyMUgyM1YzSDEwWiIgZmlsbD0iIzYyN0E3OCIvPgo8cGF0aCBkPSJNOC4xNyA1LjdMMTUgMTAuMjVWMjFIMVYxMC40OEw4LjE3IDUuN1pNMTAgMTlIMTNWMTEuMTZMOC4xNyA4LjA5TDMgMTEuMzhWMTlINlYxM0gxMFYxOVoiIGZpbGw9IiM2MjdBNzgiLz4KPC9zdmc+Cg==',
                                             'experiencia': 'data:image/svg+xml;base64,PHN2ZyB3aWR0aD0iMjAiIGhlaWdodD0iMjAiIHZpZXdCb3g9IjAgMCAyMCAyMCIgZmlsbD0ibm9uZSIgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIj4KPHBhdGggZD0iTTEyIDVIMTIuNVY0LjVWMi41VjJIMTJIOEg3LjVWMi41VjQuNVY1SDhIMTJaTTIgNkgxLjVWNi41VjE3LjVWMThIMkgxOEgxOC41VjE3LjVWNi41VjZIMThIMlpNMC41MSA2LjUwMDQ1VjYuNUMwLjUxIDUuNjYyODYgMS4xNjk0MSA1IDIgNUg2SDYuNVY0LjVWMi41QzYuNSAxLjY2NjE0IDcuMTY2MTQgMSA4IDFIMTJDMTIuODMzOSAxIDEzLjUgMS42NjYxNCAxMy41IDIuNVY0LjVWNUgxNEgxOEMxOC44MzM5IDUgMTkuNSA1LjY2NjE0IDE5LjUgNi41VjE3LjVDMTkuNSAxOC4zMzM5IDE4LjgzMzkgMTkgMTggMTlIMkMxLjE2NjIxIDE5IDAuNTAwMTExIDE4LjMzNCAwLjUgMTcuNTAwMkMwLjUgMTcuNTAwMSAwLjUgMTcuNTAwMSAwLjUgMTcuNUwwLjUxIDYuNTAwNDVaIiBmaWxsPSIjNjI3QTc4IiBzdHJva2U9IiM2MjdBNzgiLz4KPC9zdmc+Cg==',
                                             'jornada_tipo': 'data:image/svg+xml;base64,PHN2ZyB3aWR0aD0iMjQiIGhlaWdodD0iMjQiIHZpZXdCb3g9IjAgMCAyNCAyNCIgZmlsbD0ibm9uZSIgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIj4KPHBhdGggZD0iTTcgMTZIMTRWMThIN1YxNlpNNyAxMkgxN1YxNEg3VjEyWk03IDhIMTdWMTBIN1Y4Wk0xOSA0SDE0LjgyQzE0LjQgMi44NCAxMy4zIDIgMTIgMkMxMC43IDIgOS42IDIuODQgOS4xOCA0SDVDNC44NiA0IDQuNzMgNC4wMSA0LjYgNC4wNEM0LjIxIDQuMTIgMy44NiA0LjMyIDMuNTkgNC41OUMzLjQxIDQuNzcgMy4yNiA0Ljk5IDMuMTYgNS4yM0MzLjA2IDUuNDYgMyA1LjcyIDMgNlYyMEMzIDIwLjI3IDMuMDYgMjAuNTQgMy4xNiAyMC43OEMzLjI2IDIxLjAyIDMuNDEgMjEuMjMgMy41OSAyMS40MkMzLjg2IDIxLjY5IDQuMjEgMjEuODkgNC42IDIxLjk3QzQuNzMgMjEuOTkgNC44NiAyMiA1IDIySDE5QzIwLjEgMjIgMjEgMjEuMSAyMSAyMFY2QzIxIDQuOSAyMC4xIDQgMTkgNFpNMTIgMy43NUMxMi40MSAzLjc1IDEyLjc1IDQuMDkgMTIuNzUgNC41QzEyLjc1IDQuOTEgMTIuNDEgNS4yNSAxMiA1LjI1QzExLjU5IDUuMjUgMTEuMjUgNC45MSAxMS4yNSA0LjVDMTEuMjUgNC4wOSAxMS41OSAzLjc1IDEyIDMuNzVaTTE5IDIwSDVWNkgxOVYyMFoiIGZpbGw9IiM2MjdBNzgiLz4KPC9zdmc+Cg==',
                                             'idioma': 'data:image/svg+xml;base64,PHN2ZyB3aWR0aD0iMjAiIGhlaWdodD0iMjAiIHZpZXdCb3g9IjAgMCAyMCAyMCIgZmlsbD0ibm9uZSIgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIj4KPHBhdGggZD0iTTMuMDggMTMuNUgyLjIxMDYyTDIuNjQ3ODMgMTQuMjUxNEMzLjY2NzA2IDE2LjAwMzIgNS4yOTE5MSAxNy4zNjMzIDcuMjQ4MDMgMTguMDMzTDguNDU4MDEgMTguNDQ3M0w3Ljg0OTg1IDE3LjMyMjJDNy4yNzA2MSAxNi4yNTA3IDYuODI0OTYgMTUuMDg5MiA2LjUxNDM4IDEzLjg3Nkw2LjQxODEyIDEzLjVINi4wM0gzLjA4Wk05LjU4ODc4IDE4LjI0NDRMMTAgMTguODM5TDEwLjQxMTIgMTguMjQ0NEMxMS4yNzAxIDE3LjAwMjYgMTEuOTQzNCAxNS42MjU0IDEyLjM4ODggMTQuMTQ0TDEyLjU4MjUgMTMuNUgxMS45MUw4LjA5IDEzLjVINy40MTc1M0w3LjYxMTE4IDE0LjE0NEM4LjA1NjY0IDE1LjYyNTQgOC43Mjk4NiAxNy4wMDI2IDkuNTg4NzggMTguMjQ0NFpNMTguMjI1MSA3Ljg3ODczTDE4LjEzMDQgNy41SDE3Ljc0SDE0LjM2SDEzLjc5NTdMMTMuODYzNiA4LjA2MDE3QzEzLjk0MjUgOC43MTExNiAxNCA5LjM0ODc1IDE0IDEwQzE0IDEwLjY1MTIgMTMuOTQyNSAxMS4yODg4IDEzLjg2MzYgMTEuOTM5OEwxMy43OTU3IDEyLjVIMTQuMzZIMTcuNzRIMTguMTMwNEwxOC4yMjUxIDEyLjEyMTNDMTguMzkzIDExLjQ0OTUgMTguNSAxMC43MzgzIDE4LjUgMTBDMTguNSA5LjI2MTczIDE4LjM5MyA4LjU1MDUyIDE4LjIyNTEgNy44Nzg3M1pNMTYuOTIgNi41SDE3Ljc4NjdMMTcuMzUyOCA1Ljc0OTY5QzE2LjMzMjggMy45ODU5IDE0LjcwNyAyLjYzNjMzIDEyLjc1MiAxLjk2Njk2TDExLjU0MiAxLjU1MjY4TDEyLjE1MDEgMi42Nzc3NkMxMi43Mjk0IDMuNzQ5MzUgMTMuMTc1IDQuOTEwOCAxMy40ODU2IDYuMTI0TDEzLjU4MTkgNi41SDEzLjk3SDE2LjkyWk0xMy45NyAxMy41SDEzLjU4MTlMMTMuNDg1NiAxMy44NzZDMTMuMTc1IDE1LjA4OTIgMTIuNzI5NCAxNi4yNTA3IDEyLjE1MDEgMTcuMzIyMkwxMS41NDIgMTguNDQ3M0wxMi43NTIgMTguMDMzQzE0LjcwNyAxNy4zNjM3IDE2LjMzMjggMTYuMDE0MSAxNy4zNTI4IDE0LjI1MDNMMTcuNzg2NyAxMy41SDE2LjkySDEzLjk3Wk0xMC40MTEyIDEuNzU1NThMMTAgMS4xNjEwNEw5LjU4ODc4IDEuNzU1NThDOC43Mjk4NiAyLjk5NzM5IDguMDU2NjQgNC4zNzQ2MSA3LjYxMTE4IDUuODU2MDJMNy40MTc1MyA2LjVIOC4wOUgxMS45MUgxMi41ODI1TDEyLjM4ODggNS44NTYwMkMxMS45NDM0IDQuMzc0NjEgMTEuMjcwMSAyLjk5NzM5IDEwLjQxMTIgMS43NTU1OFpNNy42NiA3LjVINy4yMjM1NUw3LjE2NDU5IDcuOTMyNDRDNy4wNzM0MyA4LjYwMDkzIDcgOS4yODY3MyA3IDEwQzcgMTAuNzEzIDcuMDczMzQgMTEuNDA4NiA3LjE2NDczIDEyLjA2ODZMNy4yMjQ0NiAxMi41SDcuNjZMMTIuMzQgMTIuNUgxMi43NzU1TDEyLjgzNTMgMTIuMDY4NkMxMi45MjY3IDExLjQwODYgMTMgMTAuNzEzIDEzIDEwQzEzIDkuMjg2NzMgMTIuOTI2NiA4LjYwMDkzIDEyLjgzNTQgNy45MzI0NEwxMi43NzY0IDcuNUwxMi4zNCA3LjVINy42NlpNNy44NDk4NSAyLjY3Nzc2TDguNDU4MDEgMS41NTI2OEw3LjI0ODA0IDEuOTY2OTZDNS4yOTE5MSAyLjYzNjcyIDMuNjY3MDYgMy45OTY3NSAyLjY0NzgzIDUuNzQ4NTVMMi4yMTA2MiA2LjVIMy4wOEg2LjAzSDYuNDE4MTJMNi41MTQzOCA2LjEyNEM2LjgyNDk2IDQuOTEwOCA3LjI3MDYxIDMuNzQ5MzUgNy44NDk4NSAyLjY3Nzc2Wk02LjEzNjM3IDguMDYwMTdMNi4yMDQyNiA3LjVINS42NEgyLjI2SDEuODY5NjFMMS43NzQ5MyA3Ljg3ODczQzEuNjA2OTggOC41NTA1MiAxLjUgOS4yNjE3MyAxLjUgMTBDMS41IDEwLjczODMgMS42MDY5OCAxMS40NDk1IDEuNzc0OTMgMTIuMTIxM0wxLjg2OTYxIDEyLjVIMi4yNkg1LjY0SDYuMjA0MjZMNi4xMzYzNyAxMS45Mzk4QzYuMDU3NDYgMTEuMjg4OCA2IDEwLjY1MTIgNiAxMEM2IDkuMzQ4NzUgNi4wNTc0NiA4LjcxMTE2IDYuMTM2MzcgOC4wNjAxN1pNMTkuNSAxMEMxOS41IDE1LjI0NDUgMTUuMjUzMiAxOS41IDEwLjAxIDE5LjVDNC43NTU4MiAxOS41IDAuNSAxNS4yNDM1IDAuNSAxMEMwLjUgNC43NTY0NyA0Ljc1NTgyIDAuNSAxMC4wMSAwLjVDMTUuMjUzMiAwLjUgMTkuNSA0Ljc1NTQ5IDE5LjUgMTBaIiBmaWxsPSIjMDJBRUEyIiBzdHJva2U9IiM2MjdBNzgiLz4KPC9zdmc+Cg==',
                                             'salario': 'data:image/svg+xml;base64,PHN2ZyB3aWR0aD0iMjAiIGhlaWdodD0iMjAiIHZpZXdCb3g9IjAgMCAyMCAyMCIgZmlsbD0ibm9uZSIgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIj4KPHBhdGggZD0iTTEwLjczIDQuNzFWNS4xMDE1NkwxMS4xMTAxIDUuMTk1NDJDMTIuMjg0NCA1LjQ4NTM3IDEyLjk0OTkgNi4yNjQzMSAxMy4xNTQxIDcuMThIMTIuNDUyQzEyLjM2OTcgNi43ODU5NyAxMi4xOTU2IDYuNDE4NjYgMTEuODg0IDYuMTI3MjdDMTEuNDUxIDUuNzIyNDQgMTAuODMzMiA1LjU0IDEwLjA3IDUuNTRDOS4zNDcwMiA1LjU0IDguNzExOTUgNS43MDE5MiA4LjI0MjMgNi4wMzAxOEM3Ljc1ODkyIDYuMzY4MDUgNy40NyA2Ljg3Mjk1IDcuNDcgNy40N0M3LjQ3IDguMDAzNzMgNy42OTYwNyA4LjQ1MzU5IDguMTc4MjQgOC44MTE0OUM4LjYyMzI5IDkuMTQxODMgOS4yODY5MSA5LjM5NTY5IDEwLjE4NTEgOS42MjQxNUMxMS4wNTUgOS44NTE1OSAxMS44ODUxIDEwLjEzNzIgMTIuNDk1NCAxMC41OTY5QzEzLjA3ODIgMTEuMDM1OCAxMy40NjUgMTEuNjM0NCAxMy40NyAxMi41NTk2QzEzLjQ2NTIgMTMuMjI1MSAxMy4yMjA4IDEzLjcxMjYgMTIuODMxNSAxNC4wNzMyQzEyLjQyOTQgMTQuNDQ1NiAxMS44NDYgMTQuNzA1NCAxMS4xNDYzIDE0LjgzODlMMTAuNzQgMTQuOTE2NFYxNS4zM1YxNi41SDkuNFYxNS4zVjE0Ljg5MjhMOS4wMDExOSAxNC44MTAzQzcuODIwNDkgMTQuNTY2MyA2Ljg5ODk0IDEzLjkxMDkgNi42MjA1NiAxMi44M0g3LjM0NzA0QzcuNDYwNjIgMTMuMjI3MyA3LjY3OTg2IDEzLjU5NjIgOC4wNDE0IDEzLjg4NTRDOC41MjgxNyAxNC4yNzQ4IDkuMjEwODUgMTQuNDcgMTAuMDggMTQuNDdDMTEuMDE1MyAxNC40NyAxMS42NzU0IDE0LjIzNDIgMTIuMTA2NSAxMy44NDQ3QzEyLjUzODcgMTMuNDU0MiAxMi42OCAxMi45NjMyIDEyLjY4IDEyLjU4QzEyLjY4IDEyLjExMTcgMTIuNTQ1MiAxMS42MTYgMTIuMDgzMyAxMS4xODI4QzExLjY0ODcgMTAuNzc1MyAxMC45NjQzIDEwLjQ2MTYgOS45NTU4NSAxMC4yMjM2QzguODk3ODUgOS45NzAwMiA4LjA2Njk4IDkuNjM2NDQgNy41MDgxMyA5LjE5MDQ1QzYuOTcyMzggOC43NjI4OCA2LjY4IDguMjI5NSA2LjY4IDcuNUM2LjY4IDYuMzIzMTQgNy42MTQzNCA1LjQ3MzU2IDkuMDAzNjYgNS4xNzkxNEw5LjQgNS4wOTUxNFY0LjY5VjMuNUgxMC43M1Y0LjcxWk0wLjUgMTBDMC41IDQuNzU2MTQgNC43NTYxNCAwLjUgMTAgMC41QzE1LjI0MzkgMC41IDE5LjUgNC43NTYxNCAxOS41IDEwQzE5LjUgMTUuMjQzOSAxNS4yNDM5IDE5LjUgMTAgMTkuNUM0Ljc1NjE0IDE5LjUgMC41IDE1LjI0MzkgMC41IDEwWk0xLjUgMTBDMS41IDE0LjY4NjEgNS4zMTM4NiAxOC41IDEwIDE4LjVDMTQuNjg2MSAxOC41IDE4LjUgMTQuNjg2MSAxOC41IDEwQzE4LjUgNS4zMTM4NiAxNC42ODYxIDEuNSAxMCAxLjVDNS4zMTM4NiAxLjUgMS41IDUuMzEzODYgMS41IDEwWiIgZmlsbD0iIzYyN0E3OCIgc3Ryb2tlPSIjNjI3QTc4Ii8+Cjwvc3ZnPgo='}}
        },
     
        'tecnoempleo': {
            'busqueda_por_empleo' : False,
            'busqueda_por_url' : True,
            
            'url': 'about:blank',
            'nombres_columnas': ["titulo", "empresa", "fecha", "herramientas", "descripcion", "ubicacion", "funciones", "jornada", "experiencia", "tipo_contrato", "salario", "fecha_scrapeo", "url", "portal"],
            
            'dic_datos' : {'limite' : 1000,
                           'nuevas' : True}
        },
      
        'infojobs': {
            'busqueda_por_empleo' : False,
            'busqueda_por_url' : True,
            
            'url': 'about:blank',
            'nombres_columnas': ["titulo", "empresa", "fecha", "herramientas", "descripcion", "ubicacion", "funciones", "jornada", "experiencia", "tipo_contrato", "salario", "fecha_scrapeo", "url", "portal"],
            
            'dic_datos' : {'api_key' : None,
                           'opcion' : '_15_DAYS',
                           'opciones_fechas_publicacion' : ['_24_HOURS', '_7_DAYS', '_15_DAYS']}
        },
    },
    
    
    'empleos_a_buscar': [
            "Desarrollador de software", "Ingeniero de desarrollo", "Analista de sistemas", "Desarrollador web", "Ingeniero DevOps", "Administrador de bases de datos", "Científico de datos", "Ingeniero de aprendizaje automático", "Desarrollador de aplicaciones", "Analista de seguridad informática", "Ingeniero de redes",
            "Ingeniero de sistemas", "Desarrollador de juegos", "Analista de negocios de TI", "Arquitecto de software", "Ingeniero de pruebas de software", "Desarrollador de interfaces de usuario", "Ingeniero de automatización", "Especialista en análisis de rendimiento", "Ingeniero de realidad extendida"
            "Software Developer", "Development Engineer", "Systems Analyst", "Web Developer", "DevOps Engineer", "Database Administrator", "Data Scientist", "Machine Learning Engineer", "Applications Developer", "IT Security Analyst",
            "Network Engineer", "Systems Engineer", "Game Developer", "IT Business Analyst", "Software Architect", "Software Test Engineer", "UI/UX Developer", "Automation Engineer", "Performance Analyst", "Extended Reality Engineer"
        ],
}

# Convertir el diccionario a formato JSON
json_data = json.dumps(diccionario_datos, indent=4)

# Guardar el JSON en un archivo (opcional)
with open('datos_scrapers.json', 'w') as json_file:
    json_file.write(json_data)